<a href="https://colab.research.google.com/github/Shahbaz67/Skill_taxonomy/blob/main/Skill_Taxonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from glove import Corpus, Glove
import PyPDF2 as pdf
import re
import numpy as np
from gensim.models import Word2Vec
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords, stem_text
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file = open('/NCO - 2015_Full.pdf', 'rb')
pdf_reader = pdf.PdfFileReader(file)

In [ ]:
skill_set = ['researching', 'analysing', 'managing','evaluating', 'planning', 'negotiating', 'coordinating', 'organising', 'teaching' 'selling', 'advertising', 'calculating', 'bookkeeping', 'correcting data', 'operating', 'repairing', 'serving']

In [ ]:
pdf_reader.numPages

1519

In [ ]:
# making the list of occupation description
def Dataset(pdf_reader):
    start = 367
    i = 0
    occ_code = []
    occ_description = []
    page = []
    text = ""
    while(start != 1519):
        page = pdf_reader.getPage(start)
        page_extract = page.extractText()
        new_page = page_extract
        new_page = new_page.replace("\n", "")
        new_page = re.sub(' +', ' ', new_page)
        new_page = re.sub(re.findall(r'[A-Z\s-]*\d{4}\s[A-Z\s]*\d{3,4}\s', new_page)[0], "", new_page)
        text += new_page
        start += 1
        i += 1
    res = re.compile(r'(\d{4}\.\d{4})(.*?(?=ISCO))')
    group = re.findall(res, text)
    for x in group:
        occ_code.append(x[0])
        occ_description.append(x[1])
    print("Number of occupations = {}".format(len(group)))
    # print("ISCO : {}".format(len(re.findall(r'ISCO',text))))
    return (occ_code, occ_description)

In [ ]:
occ_code, occ_description = Dataset(pdf_reader)

Number of occupations = 3447


In [ ]:
# Text cleaning using nltk
sentences = []
for w in occ_description:
    words = w.lower()
    words = re.sub('[\.,-:;\'/\\)\\(]', ' ', words)
    words = remove_stopwords(words)
    wnl = WordNetLemmatizer()
    words = wnl.lemmatize(words)
    sentences.append(words.split())

In [ ]:
# Word2vec
model = Word2Vec(sentences = sentences, size = 100, sg = 1, window = 3, min_count = 1, iter = 10, workers = 3)

In [ ]:
model.save('word2vec_model')
model = Word2Vec.load('word2vec_model')

In [ ]:
model.most_similar('researching')

[('opinions', 0.9876055717468262),
 ('era', 0.9862070083618164),
 ('prioritise', 0.9861675500869751),
 ('correlations', 0.985716700553894),
 ('techno', 0.9854756593704224),
 ('urges', 0.9847025871276855),
 ('grievance', 0.9845303893089294),
 ('strenuous', 0.9843680262565613),
 ('courseware', 0.984313428401947),
 ('organismic', 0.984301745891571)]